In [2]:
# Convert featureTable into tabular data

In [1]:
import pandas as pd

In [2]:
path = "data/refseq/GCF_000007565.2_ASM756v2_feature_table.txt.gz"
df = pd.read_table(path)

df.drop(columns = ['assembly', 'assembly_unit', 'seq_type', 'chromosome', 'genomic_accession'], inplace=True)

print(df.head())

  # feature           class  start   end strand product_accession  \
0      gene  protein_coding    147  1019      -               NaN   
1       CDS    with_protein    147  1019      -    WP_003253185.1   
2      gene  protein_coding   1029  1820      -               NaN   
3       CDS    with_protein   1029  1820      -    WP_003253182.1   
4      gene  protein_coding   1839  2489      -               NaN   

  non-redundant_refseq  related_accession  \
0                  NaN                NaN   
1       WP_003253185.1                NaN   
2                  NaN                NaN   
3       WP_003253182.1                NaN   
4                  NaN                NaN   

                                       name symbol  GeneID   locus_tag  \
0                                       NaN    NaN     NaN  PP_RS00005   
1  ParB/RepB/Spo0J family partition protein    NaN     NaN  PP_RS00005   
2                                       NaN    NaN     NaN  PP_RS00010   
3                 

In [3]:
gene = df[df['# feature'].str.contains('gene')]
product = df[df['# feature'].str.contains('gene') == False]

wide = gene.merge(product, on=['locus_tag', 'start', 'end', 'strand', 'symbol', 'feature_interval_length'], how='outer',suffixes=['_gene','_product'])

# Identify constant columns and remove them
constant_columns = [col for col in wide.columns if wide[col].isnull().all()]
wide.drop(labels = constant_columns, axis = 1, inplace=True)

# separat
wide[['attributes_gene', 'old_locus_tag']] = wide['attributes_gene'].str.split("=", expand = True)

# remove forbidden characters (#) from the column names
wide.columns = wide.columns.str.replace(r'# ', '')

# Display constant columns
print(wide)



     feature_gene      class_gene    start      end strand symbol   locus_tag  \
0            gene  protein_coding      147     1019      -    NaN  PP_RS00005   
1            gene  protein_coding     1029     1820      -    NaN  PP_RS00010   
2            gene  protein_coding     1839     2489      -   rsmG  PP_RS00015   
3            gene  protein_coding     2486     4378      -   mnmG  PP_RS00020   
4            gene  protein_coding     5012     6382      -   mnmE  PP_RS00025   
...           ...             ...      ...      ...    ...    ...         ...   
5692         gene  protein_coding  6181463  6181870      -    NaN  PP_RS28245   
5693          NaN             NaN  1700460  1701555      +   prfB  PP_RS07705   
5694          NaN             NaN  3376573  3377786      +    NaN  PP_RS15420   
5695          NaN             NaN  5000914  5001769      +    NaN  PP_RS22875   
5696          NaN             NaN  5014190  5015352      +    NaN  PP_RS22940   

      feature_interval_leng

In [4]:
wide.to_csv("output/refseq.tsv", sep="\t", index=False)